In [37]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

In [38]:


#dosyayı oku
df = pd.read_excel("output_cleaned_no_empty_rows.xlsx") 

# NLTK kütüphanesinden Vader duygusal analiz sözlüğünü indirin
nltk.download('vader_lexicon')

# SentimentIntensityAnalyzer sınıfını oluşturun
sentiments = SentimentIntensityAnalyzer()

# Duygusal skorları hesaplayarak veri çerçevesine ekleyin
df["Positive"] = [sentiments.polarity_scores(comment)["pos"] for comment in df["Cümle"]]
df["Negative"] = [sentiments.polarity_scores(comment)["neg"] for comment in df["Cümle"]]
df["Neutral"] = [sentiments.polarity_scores(comment)["neu"] for comment in df["Cümle"]]
df['Compound'] = [sentiments.polarity_scores(comment)["compound"] for comment in df["Cümle"]]

# Duygusal skorlara dayalı olarak etiketleme yapın
score = df["Compound"].values
sentiment = []

for i in score:
    if i >= 0.05 :
        sentiment.append('Positive')
    elif i <= -0.05 :
        sentiment.append('Negative')
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
df.head()


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\tahat\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Cümle,Positive,Negative,Neutral,Compound,Sentiment
0,watched love christopher nolan movies mannnn g...,0.323,0.098,0.579,0.8779,Positive
1,often found bored trying desperately understan...,0.224,0.376,0.400,-0.2263,Negative
2,id like consider fairly intelligent person cou...,0.440,0.000,0.560,0.6705,Positive
3,hahaha,1.000,0.000,0.000,0.5574,Positive
4,agree people didnt actually expect nolan take ...,0.326,0.055,0.619,0.8519,Positive


In [39]:
pd.options.display.max_rows = None

In [40]:

data2=df.drop(['Positive','Negative','Neutral','Compound'],axis=1)

In [41]:
data2.to_excel("labelingData.xlsx" , index=False, engine='openpyxl')

In [42]:
nltk.download('omw-1.4')
data_copy = data2.copy()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tahat\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [43]:
le = LabelEncoder()
data_copy['Sentiment'] = le.fit_transform(data_copy['Sentiment'])

In [44]:
processed_data = {
    'Sentence': data_copy['Cümle'],  # Eğer sütun adı "Cümle" ise, doğru adı kullanmalısınız.
    'Sentiment': data_copy['Sentiment']
}

processed_data = pd.DataFrame(processed_data)
processed_data.head()


,Sentence,Sentiment
0,watched love christopher nolan movies mannnn g...,2
1,often found bored trying desperately understan...,0
2,id like consider fairly intelligent person cou...,2
3,hahaha,2
4,agree people didnt actually expect nolan take ...,2


In [45]:
processed_data['Sentiment'].value_counts()

Sentiment
2    5231
1    3411
0    3102
Name: count, dtype: int64

In [46]:
processed_data['Sentiment'].value_counts()

Sentiment
2    5231
1    3411
0    3102
Name: count, dtype: int64

In [47]:
corpus = []
for sentence in processed_data['Sentence']:
    corpus.append(sentence)

In [48]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = processed_data.iloc[:, -1].values

In [49]:
#model eğitimi
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [50]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
cm

array([[440, 393,  71],
       [ 47, 948,  31],
       [403, 773, 418]], dtype=int64)

In [51]:
nb_score = accuracy_score(y_test, y_pred)
print('accuracy',nb_score)

accuracy 0.5124858115777525


In [52]:
# Örnek bir yorumu tahmin etme
example_comment = ["still good movie wasnt horrifying visually awesome hoping"]
example_comment_transformed = cv.transform(example_comment).toarray()
prediction = classifier.predict(example_comment_transformed)

# Tahmin sonucunu yazdırma
print('Tahmin Sonucu:', le.inverse_transform(prediction))

Tahmin Sonucu: ['Negative']


In [56]:
# Örnek bir yorumu tahmin etme
example_comment = ["boring hard keep since kept switching around past present future"]
example_comment_transformed = cv.transform(example_comment).toarray()
prediction = classifier.predict(example_comment_transformed)

# Tahmin sonucunu yazdırma
print('Tahmin Sonucu:', le.inverse_transform(prediction))

Tahmin Sonucu: ['Negative']
